# StoryLand AI

**Transform books into travel adventures.**

This notebook demonstrates a multi-agent system built with Google's Agent Development Kit that automatically creates personalized travel itineraries from your favorite books. Simply provide a book title, and watch as specialized AI agents work together to research locations, discover landmarks, and compose a complete journey through the story's world.

## Setup

In [99]:
# Import libraries
import os
import sys
import json
import time
import requests
import uuid
import logging
from typing import List, Optional, Dict, Any
from dataclasses import dataclass
from pathlib import Path
from dotenv import load_dotenv

# Import Google ADK components
from google.genai import types
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import FunctionTool, google_search, AgentTool

# Import logging
from common.logging import setup_logger, get_tracer

# Load environment
load_dotenv()

# Enable DEBUG logging to see step-by-step execution
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(name)s - %(message)s'
)

# Setup logging
logger = setup_logger("storyland_demo")
tracer = get_tracer()

print("Environment Setup:")
print(f"   Google API Key: {'OK' if os.getenv('GOOGLE_API_KEY') else 'MISSING'}")
print(f"   Log Level: DEBUG (step-by-step execution visible)")

Environment Setup:
   Google API Key: OK
   Log Level: DEBUG (step-by-step execution visible)


## Pydantic Data Models

Define type-safe data structures for explicit agent-to-agent communication.

These models ensure:
- **Type safety** - Automatic validation of data types
- **Clear contracts** - Each agent knows exactly what data to expect/produce
- **Better debugging** - Validation errors caught early with detailed messages
- **IDE support** - Autocomplete and type hints

In [100]:
from pydantic import BaseModel, Field
from typing import Optional, List

# Book metadata from Google Books API
class BookMetadata(BaseModel):
    """Book metadata from Google Books API"""
    book_title: str = Field(description="Full book title")
    author: str = Field(description="Primary author name")
    description: Optional[str] = Field(default=None, description="Book description/synopsis")
    published_date: Optional[str] = Field(default=None, description="Publication date")
    categories: List[str] = Field(default_factory=list, description="Book categories/genres")
    image_url: Optional[str] = Field(default=None, description="Book cover image URL")

# Book context (setting, time period, themes)
class BookContext(BaseModel):
    """Book setting and context information"""
    primary_locations: List[str] = Field(description="Main locations where story takes place")
    time_period: str = Field(description="Historical era or time period of the story")
    themes: List[str] = Field(description="Main themes of the book")

# City discovery
class CityInfo(BaseModel):
    """Information about a city to visit"""
    name: str = Field(description="City name")
    country: str = Field(description="Country name")
    relevance: str = Field(description="How this city relates to the book")

class CityDiscovery(BaseModel):
    """Discovery results for cities"""
    cities: List[CityInfo] = Field(description="List of cities related to the book")

# Landmark discovery
class LandmarkInfo(BaseModel):
    """Information about a landmark"""
    name: str = Field(description="Landmark or place name")
    city: str = Field(description="City where landmark is located")
    connection: str = Field(description="How this landmark relates to the book")

class LandmarkDiscovery(BaseModel):
    """Discovery results for landmarks"""
    landmarks: List[LandmarkInfo] = Field(description="List of landmarks related to the book")

# Author sites
class AuthorSiteInfo(BaseModel):
    """Information about author-related sites"""
    name: str = Field(description="Site name")
    type: str = Field(description="Type of site (museum, birthplace, etc.)")
    city: str = Field(description="City where site is located")

class AuthorSites(BaseModel):
    """Discovery results for author-related sites"""
    author_sites: List[AuthorSiteInfo] = Field(description="List of author-related sites")

# Travel itinerary
class CityStop(BaseModel):
    """A stop/place to visit in a city"""
    name: str = Field(description="Name of the place")
    type: str = Field(description="Type: landmark, museum, cafe, restaurant, bookstore, etc.")
    reason: str = Field(description="Why this place matters for the book")
    time_of_day: str = Field(description="Best time to visit: morning, afternoon, evening, full_day")
    notes: Optional[str] = Field(default=None, description="Practical tips for visiting")

class CityPlan(BaseModel):
    """Travel plan for a specific city"""
    name: str = Field(description="City name")
    country: str = Field(description="Country name")
    days_suggested: int = Field(description="Suggested number of days to spend", ge=1, le=7)
    overview: str = Field(description="Brief overview of what to expect in this city")
    stops: List[CityStop] = Field(description="Places to visit in this city")

class TripItinerary(BaseModel):
    """Complete travel itinerary"""
    cities: List[CityPlan] = Field(description="City-by-city travel plans")
    summary_text: str = Field(description="Engaging overview of the entire trip")

print("✅ Pydantic models defined:")
print("   - BookMetadata, BookContext")
print("   - CityDiscovery, LandmarkDiscovery, AuthorSites")
print("   - TripItinerary (with CityPlan and CityStop)")

✅ Pydantic models defined:
   - BookMetadata, BookContext
   - CityDiscovery, LandmarkDiscovery, AuthorSites
   - TripItinerary (with CityPlan and CityStop)


#### Define BookInfo Data Structure

Create a dataclass to hold book information returned from Google Books API.

In [101]:
class BookInfo(BaseModel):
    """Book information from Google Books API (internal use)"""
    title: str
    authors: List[str]
    description: Optional[str] = None
    published_date: Optional[str] = None
    categories: List[str] = Field(default_factory=list)
    image_url: Optional[str] = Field(default=None, description="Book cover image URL")

print("✅ Created BookInfo Pydantic model")

✅ Created BookInfo Pydantic model


#### Google Books API Search

Function that calls the Google Books API to search for books and parse the results.

In [102]:
def search_books(title: str, author: Optional[str] = None, max_results: int = 5) -> List[BookInfo]:
    """
    Search Google Books API and return matching books.
    
    Args:
        title: Book title to search for
        author: Optional author name
        max_results: Maximum number of results
        
    Returns:
        List of BookInfo objects
    """
    logger.info(f"Searching Google Books for: {title}")
    
    try:
        # Build query
        query_parts = []
        if title:
            query_parts.append(f"intitle:{title}")
        if author:
            query_parts.append(f"inauthor:{author}")
        
        query = "+".join(query_parts)
        
        # API request
        url = "https://www.googleapis.com/books/v1/volumes"
        params = {
            "q": query,
            "maxResults": max_results,
            "printType": "books"
        }
        
        print(f"Searching: {query}")
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        
        data = response.json()
        items = data.get("items", [])
        
        # Parse results
        results = []
        for item in items:
            vol = item.get("volumeInfo", {})
            
            # Extract image URL (prefer larger sizes)
            image_links = vol.get("imageLinks", {})
            image_url = (
                image_links.get("large") or
                image_links.get("medium") or
                image_links.get("thumbnail") or 
                image_links.get("smallThumbnail") or
                None
            )
            
            results.append(BookInfo(
                title=vol.get("title", "Unknown"),
                authors=vol.get("authors", []),
                description=vol.get("description"),
                published_date=vol.get("publishedDate"),
                categories=vol.get("categories", []),
                image_url=image_url
            ))
        
        logger.info(f"Found {len(results)} books")
        print(f"Found {len(results)} books")
        
        # Log all found books
        for i, book in enumerate(results):
            author_str = ", ".join(book.authors) if book.authors else "Unknown"
            print(f"   [{i}] {book.title} by {author_str} ({book.published_date or 'N/A'})")
        
        return results
        
    except Exception as e:
        logger.error(f"Google Books search failed: {str(e)}")
        raise

print("Created search_books() function")

Created search_books() function


#### Create FunctionTool for Book Search

**Simple Search Pattern** - Agent searches and gets the first/best match:

1. **Agent calls tool** with book title (e.g., "nightingale")
2. **Tool searches** Google Books API → Finds matching books
3. **Tool logs** all books found for visibility
4. **Tool returns** the first/best match automatically
5. **Agent continues** with the selected book

This is a streamlined approach where the tool automatically selects the most relevant book.

In [103]:
def search_book(title: str, author: str = "") -> str:
    """
    Search Google Books API and return Pydantic-validated book metadata.
    
    This tool searches for books, automatically selects the best match,
    and returns JSON validated against the BookMetadata Pydantic schema.
    
    Args:
        title: Book title to search for (e.g., "The Nightingale", "1984")
        author: Optional author name (e.g., "Kristin Hannah")
    
    Returns:
        JSON string conforming to BookMetadata schema
    """
    logger.info(f"search_book called: title='{title}', author='{author}'")
    
    try:
        # Search for books
        books = search_books(title=title, author=author or None, max_results=5)
        
        if not books:
            logger.warning(f"No books found for: {title}")
            return json.dumps({"error": "No books found", "query": {"title": title, "author": author}})
        
        # Select the first/best match
        selected = books[0]
        author_str = ", ".join(selected.authors) if selected.authors else "Unknown"
        
        print(f"✅ Selected: {selected.title} by {author_str}")
        logger.info(f"Selected book: {selected.title} by {author_str}")
        
        # Create and validate with Pydantic
        book_metadata = BookMetadata(
            book_title=selected.title,
            author=author_str,
            description=selected.description,
            published_date=selected.published_date,
            categories=selected.categories or [],
            image_url=selected.image_url
        )
        
        # Return Pydantic-validated JSON
        return book_metadata.model_dump_json()
        
    except Exception as e:
        logger.error(f"Book search failed: {str(e)}", exc_info=True)
        return json.dumps({"error": str(e), "type": type(e).__name__})

# Create FunctionTool
google_books_tool = FunctionTool(search_book)

print("✅ Created FunctionTool:")
print("   Function: search_book")
print("   Returns: Pydantic-validated BookMetadata JSON")

✅ Created FunctionTool:
   Function: search_book
   Returns: Pydantic-validated BookMetadata JSON


## Model Configuration

In [104]:
# Configure Gemini model
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504]
)

model = Gemini(
    model="gemini-2.0-flash-lite",
    api_key=os.getenv("GOOGLE_API_KEY"),
    retry_options=retry_config
)

print(f"Model configured: {model.model}")
print(f"Note: Two-stage pattern used (tools + Pydantic work separately)")

Model configured: gemini-2.0-flash-lite
Note: Two-stage pattern used (tools + Pydantic work separately)


## Two-Stage Agent Pattern

Build agents using a two-stage pattern:
1. **Stage 1 (Tool Agent)**: Uses tools to gather data, returns JSON
2. **Stage 2 (Formatter Agent)**: Validates and formats with Pydantic `output_schema`

This pattern works around the limitation where some models don't support tools + output_schema together.

In [105]:
print("Creating Two-Stage Agent Pipelines...")
print()

# =============================================================================
# BOOK METADATA PIPELINE
# =============================================================================
# Stage 1: Tool agent
book_metadata_researcher = LlmAgent(
    name="book_metadata_researcher",
    model=model,
    tools=[google_books_tool],
    instruction="""Extract book metadata for the book provided by the user.
Use search_book to get book information and return it."""
)

# Stage 2: Pydantic formatter
book_metadata_formatter = LlmAgent(
    name="book_metadata_formatter",
    model=model,
    output_schema=BookMetadata,
    output_key="book_metadata",
    instruction="""Format the book metadata from the previous response into validated BookMetadata."""
)

# Pipeline
book_metadata_pipeline = SequentialAgent(
    name="book_metadata_pipeline",
    sub_agents=[book_metadata_researcher, book_metadata_formatter]
)
print("   ✓ book_metadata_pipeline (2-stage)")

# =============================================================================
# BOOK CONTEXT PIPELINE
# =============================================================================
book_context_researcher = LlmAgent(
    name="book_context_researcher",
    model=model,
    tools=[google_search],
    instruction="""Research the book's setting and context using google_search.
Find: primary locations, time period, and main themes."""
)

book_context_formatter = LlmAgent(
    name="book_context_formatter",
    model=model,
    output_schema=BookContext,
    output_key="book_context",
    instruction="""Format the research into validated BookContext."""
)

book_context_pipeline = SequentialAgent(
    name="book_context_pipeline",
    sub_agents=[book_context_researcher, book_context_formatter]
)
print("   ✓ book_context_pipeline (2-stage)")

# =============================================================================
# CITY DISCOVERY PIPELINE
# =============================================================================
city_researcher = LlmAgent(
    name="city_researcher",
    model=model,
    tools=[google_search],
    instruction="""Find cities to visit based on the book using google_search."""
)

city_formatter = LlmAgent(
    name="city_formatter",
    model=model,
    output_schema=CityDiscovery,
    output_key="city_discovery",
    instruction="""Format the cities into validated CityDiscovery."""
)

city_pipeline = SequentialAgent(
    name="city_pipeline",
    sub_agents=[city_researcher, city_formatter]
)
print("   ✓ city_pipeline (2-stage)")

# =============================================================================
# LANDMARK DISCOVERY PIPELINE
# =============================================================================
landmark_researcher = LlmAgent(
    name="landmark_researcher",
    model=model,
    tools=[google_search],
    instruction="""Find landmarks related to the book using google_search."""
)

landmark_formatter = LlmAgent(
    name="landmark_formatter",
    model=model,
    output_schema=LandmarkDiscovery,
    output_key="landmark_discovery",
    instruction="""Format the landmarks into validated LandmarkDiscovery."""
)

landmark_pipeline = SequentialAgent(
    name="landmark_pipeline",
    sub_agents=[landmark_researcher, landmark_formatter]
)
print("   ✓ landmark_pipeline (2-stage)")

# =============================================================================
# AUTHOR SITES PIPELINE
# =============================================================================
author_researcher = LlmAgent(
    name="author_researcher",
    model=model,
    tools=[google_search],
    instruction="""Find author-related sites using google_search."""
)

author_formatter = LlmAgent(
    name="author_formatter",
    model=model,
    output_schema=AuthorSites,
    output_key="author_sites",
    instruction="""Format the author sites into validated AuthorSites."""
)

author_pipeline = SequentialAgent(
    name="author_pipeline",
    sub_agents=[author_researcher, author_formatter]
)
print("   ✓ author_pipeline (2-stage)")

# =============================================================================
# TRIP COMPOSER (No tools, just Pydantic)
# =============================================================================
trip_composer_agent = LlmAgent(
    name="trip_composer",
    model=model,
    output_schema=TripItinerary,
    output_key="final_itinerary",
    instruction="""Create a literary travel itinerary.

Use all the book information, cities, landmarks, and author sites from the conversation history above.

Synthesize into a cohesive travel plan with:
- Grouped stops by city
- 3-7 stops per city
- Appropriate time of day
- Specific connections to the book
- Practical visiting tips

Return validated TripItinerary."""
)
print("   ✓ trip_composer_agent (Pydantic only)")

print(f" Created two-stage pipelines with Pydantic validation")
print(f"   Each pipeline: Tool agent → Pydantic formatter")
print(f"   All outputs validated and saved to state")

Creating Two-Stage Agent Pipelines...

   ✓ book_metadata_pipeline (2-stage)
   ✓ book_context_pipeline (2-stage)
   ✓ city_pipeline (2-stage)
   ✓ landmark_pipeline (2-stage)
   ✓ author_pipeline (2-stage)
   ✓ trip_composer_agent (Pydantic only)
 Created two-stage pipelines with Pydantic validation
   Each pipeline: Tool agent → Pydantic formatter
   All outputs validated and saved to state


### Compose Workflow with Pydantic Pipelines

**Architecture with Explicit State Communication:**
```
SequentialAgent (overall_workflow)
├─ book_metadata_pipeline [fetch → format] → state["book_metadata"]
├─ book_context_pipeline [research → format] → state["book_context"]
├─ ParallelAgent (parallel_discovery) ⚡ CONCURRENT
│  ├─ city_pipeline [research → format] → state["city_discovery"]
│  ├─ landmark_pipeline [research → format] → state["landmark_discovery"]
│  └─ author_pipeline [research → format] → state["author_sites"]
└─ trip_composer_agent → state["final_itinerary"]

Each pipeline uses:
  Stage 1: Tool-using agent (saves raw data to state)
  Stage 2: Formatting agent (validates with Pydantic output_schema)
```

In [106]:
print("📦 Creating Two-Stage Agent Pipelines...")
print()

# =============================================================================
# BOOK METADATA PIPELINE
# =============================================================================
# Stage 1: Tool agent
book_metadata_researcher = LlmAgent(
    name="book_metadata_researcher",
    model=model,
    tools=[google_books_tool],
    instruction="""You are a book metadata specialist. Extract complete book information.

1. Use the search_book tool with the book title provided by the user
2. Return ALL the information you receive from the tool
3. Do not summarize or modify the data - pass it through completely"""
)

# Stage 2: Pydantic formatter
book_metadata_formatter = LlmAgent(
    name="book_metadata_formatter",
    model=model,
    output_schema=BookMetadata,
    output_key="book_metadata",
    instruction="""Format the book metadata into a validated BookMetadata object.

Extract from the previous response:
- book_title: The full book title
- author: Primary author name
- description: Book synopsis/description
- published_date: Publication date
- categories: List of genres/categories
- image_url: Cover image URL

Return a properly structured BookMetadata object."""
)

# Pipeline
book_metadata_pipeline = SequentialAgent(
    name="book_metadata_pipeline",
    sub_agents=[book_metadata_researcher, book_metadata_formatter]
)
print("   ✓ book_metadata_pipeline (2-stage)")

# =============================================================================
# BOOK CONTEXT PIPELINE
# =============================================================================
book_context_researcher = LlmAgent(
    name="book_context_researcher",
    model=model,
    tools=[google_search],
    instruction="""You are a literary research specialist. Research the book's setting and themes.

The book information is in the conversation history above.

Use google_search to find:
1. PRIMARY LOCATIONS: Where does the story take place? (cities, countries, regions)
2. TIME PERIOD: When does it take place? (specific year, era, or historical period)
3. MAIN THEMES: What are the central themes? (war, love, survival, etc.)

Search queries to try:
- "[book title] setting location"
- "[book title] time period historical context"
- "[book title] themes analysis"

Provide detailed findings with specific locations and time periods."""
)

book_context_formatter = LlmAgent(
    name="book_context_formatter",
    model=model,
    output_schema=BookContext,
    output_key="book_context",
    instruction="""Format the research into a validated BookContext object.

Extract:
- primary_locations: List of specific locations (cities, countries, regions)
- time_period: The historical era or specific timeframe
- themes: List of main themes from the book

Be specific and factual. Use proper names for locations."""
)

book_context_pipeline = SequentialAgent(
    name="book_context_pipeline",
    sub_agents=[book_context_researcher, book_context_formatter]
)
print("   ✓ book_context_pipeline (2-stage)")

# =============================================================================
# CITY DISCOVERY PIPELINE
# =============================================================================
city_researcher = LlmAgent(
    name="city_researcher",
    model=model,
    tools=[google_search],
    instruction="""You are a literary travel specialist. Find real cities readers can visit.

Based on the book's setting from the conversation history, use google_search to find:

1. SETTING CITIES: Actual cities where the story takes place
2. FILMING LOCATIONS: If adapted to film, where was it filmed?
3. AUTHOR'S CITIES: Where did the author live or get inspiration?

Search queries to try:
- "[book title] real locations to visit"
- "[book title] filming locations"
- "[book title] [author name] inspiration places"
- "[setting location] literary tourism"

For EACH city found, explain:
- What is the city name and country?
- How does it relate to the book? (setting, filming, author connection)

Find at least 2-3 cities."""
)

city_formatter = LlmAgent(
    name="city_formatter",
    model=model,
    output_schema=CityDiscovery,
    output_key="city_discovery",
    instruction="""Format the cities into validated CityDiscovery.

For each city, create a CityInfo with:
- name: City name only (e.g., "Paris", not "Paris, France")
- country: Country name (e.g., "France")
- relevance: One sentence explaining the connection to the book

Return CityDiscovery with a list of cities."""
)

city_pipeline = SequentialAgent(
    name="city_pipeline",
    sub_agents=[city_researcher, city_formatter]
)
print("   ✓ city_pipeline (2-stage)")

# =============================================================================
# LANDMARK DISCOVERY PIPELINE
# =============================================================================
landmark_researcher = LlmAgent(
    name="landmark_researcher",
    model=model,
    tools=[google_search],
    instruction="""You are a landmark research specialist. Find specific places to visit.

Based on the book and cities from the conversation history, use google_search to find:

1. MENTIONED LANDMARKS: Specific buildings, museums, or places mentioned in the book
2. THEMED EXPERIENCES: Literary walks, museum exhibits, or tours related to the book
3. ATMOSPHERIC LOCATIONS: Places that capture the book's setting or mood

Search queries to try:
- "[book title] landmarks mentioned"
- "[city name] [book title] tour"
- "[book title] museum exhibit"
- "[setting] places from [book title]"

For EACH landmark, provide:
- Exact name of the place
- Which city it's in
- Specific connection to the book (Was it mentioned? Does it relate to a scene?)

Find at least 3-5 landmarks across the cities."""
)

landmark_formatter = LlmAgent(
    name="landmark_formatter",
    model=model,
    output_schema=LandmarkDiscovery,
    output_key="landmark_discovery",
    instruction="""Format the landmarks into validated LandmarkDiscovery.

For each landmark, create a LandmarkInfo with:
- name: Exact name of the landmark or place
- city: City where it's located
- connection: One sentence explaining how it relates to the book

Return LandmarkDiscovery with a list of landmarks."""
)

landmark_pipeline = SequentialAgent(
    name="landmark_pipeline",
    sub_agents=[landmark_researcher, landmark_formatter]
)
print("   ✓ landmark_pipeline (2-stage)")

# =============================================================================
# AUTHOR SITES PIPELINE
# =============================================================================
author_researcher = LlmAgent(
    name="author_researcher",
    model=model,
    tools=[google_search],
    instruction="""You are an author biography specialist. Find places connected to the author.

Based on the author from the conversation history, use google_search to find:

1. AUTHOR'S HOMETOWN: Where was the author born or raised?
2. AUTHOR MUSEUMS: Are there museums or houses dedicated to the author?
3. WRITING LOCATIONS: Where did the author write this book?
4. MEMORIALS: Statues, plaques, or commemorative sites

Search queries to try:
- "[author name] birthplace"
- "[author name] museum"
- "[author name] house"
- "[author name] hometown"
- "where did [author name] write [book title]"

For EACH site, provide:
- Exact name
- Type (museum, birthplace, statue, etc.)
- Which city it's in

Find at least 2-3 author-related sites."""
)

author_formatter = LlmAgent(
    name="author_formatter",
    model=model,
    output_schema=AuthorSites,
    output_key="author_sites",
    instruction="""Format the author sites into validated AuthorSites.

For each site, create an AuthorSiteInfo with:
- name: Exact name of the site
- type: Type of site (e.g., "museum", "birthplace", "statue", "house")
- city: City where it's located

Return AuthorSites with a list of author_sites."""
)

author_pipeline = SequentialAgent(
    name="author_pipeline",
    sub_agents=[author_researcher, author_formatter]
)
print("   ✓ author_pipeline (2-stage)")

# =============================================================================
# TRIP COMPOSER (No tools, just Pydantic)
# =============================================================================
trip_composer_agent = LlmAgent(
    name="trip_composer",
    model=model,
    output_schema=TripItinerary,
    output_key="final_itinerary",
    instruction="""You are a literary travel planner. Create a complete travel itinerary.

Review ALL the information from the conversation history:
- Book metadata and context
- Cities to visit
- Landmarks and experiences
- Author-related sites

Create a TripItinerary with:

1. GROUP by city: Organize all stops by the city they're in
2. For EACH city, create a CityPlan with:
   - name: City name
   - country: Country name
   - days_suggested: 1-3 days (based on number of stops)
   - overview: 2-3 sentences about what to expect in this city
   - stops: 3-7 places to visit (combine landmarks and author sites)

3. For EACH stop, create a CityStop with:
   - name: Exact name of the place
   - type: "museum", "landmark", "cafe", "bookstore", "monument", etc.
   - reason: 1-2 sentences explaining WHY this matters for the book
   - time_of_day: "morning", "afternoon", "evening", or "full_day"
   - notes: Practical tip (hours, how to get there, what to look for)

4. Write a summary_text: 3-4 sentences capturing the essence of the entire journey

Make it inspiring and actionable. Readers should know exactly what to do."""
)
print("   ✓ trip_composer_agent (Pydantic only)")

print(f"\n✅ Created two-stage pipelines with precise instructions")
print(f"   Each agent has clear search guidance and output requirements")
print(f"   All outputs validated with Pydantic")

📦 Creating Two-Stage Agent Pipelines...

   ✓ book_metadata_pipeline (2-stage)
   ✓ book_context_pipeline (2-stage)
   ✓ city_pipeline (2-stage)
   ✓ landmark_pipeline (2-stage)
   ✓ author_pipeline (2-stage)
   ✓ trip_composer_agent (Pydantic only)

✅ Created two-stage pipelines with precise instructions
   Each agent has clear search guidance and output requirements
   All outputs validated with Pydantic


### Configure Your Book

Change the book title and author below, then run the cells to create a travel itinerary!

In [107]:
# ============================================================
# CONFIGURE YOUR BOOK HERE
# ============================================================
BOOK_TITLE = "gone with the wind"
AUTHOR = ""  # Optional - leave empty if unknown
# ============================================================

print(f"Book: {BOOK_TITLE}")
print(f"Author: {AUTHOR or 'Unknown'}")
print(f"\nConfiguration set!")

Book: gone with the wind
Author: Unknown

Configuration set!


### Step 3: Set Up Runner and Session

Create a runner and session to execute the workflow.

In [108]:
print("Setting up Runner and Session...")
print()

# Create session service
session_service = InMemorySessionService()
print("   - Created InMemorySessionService")

# Create runner with the workflow
runner = Runner(
    agent=workflow,
    app_name="storyland",
    session_service=session_service
)
print("   - Created Runner with workflow")

# Create a new session
session_id = str(uuid.uuid4())
await session_service.create_session(
    app_name="storyland",
    user_id="user1",
    session_id=session_id,
    state={"book_title": BOOK_TITLE, "author": AUTHOR}
)
print(f"   - Created session: {session_id[:8]}...")

print(f"\nReady to execute!")

2025-11-20 21:17:20,931 - WARNING - google_adk.google.adk.runners - App name mismatch detected. The runner is configured with app name "storyland", but the root agent was loaded from "/Users/osa/Documents/storyland/storyland-ai/.venv/lib/python3.12/site-packages/google/adk/agents", which implies app name "agents".


Setting up Runner and Session...

   - Created InMemorySessionService
   - Created Runner with workflow
   - Created session: dc7802d5...

Ready to execute!


### Step 4: Execute Workflow

Run the orchestrated workflow and collect the final result.

In [109]:
import time

print("="*70)
print("EXECUTING WORKFLOW")
print("="*70)
print(f"Book: {BOOK_TITLE}")
print(f"Author: {AUTHOR or 'Unknown'}\n")

orchestration_start = time.time()

# Create the prompt
prompt = f"""Create a literary travel itinerary for "{BOOK_TITLE}" by {AUTHOR}.

Execute all steps and return the complete combined results."""

user_message = types.Content(
    role='user',
    parts=[types.Part(text=prompt)]
)

print(f"USER REQUEST:")
print(f"{prompt}\n")
print("Starting workflow execution...\n")
print("="*70)

# Run workflow and observe events
final_response = None
event_count = 0

async for event in runner.run_async(
    user_id="user1",
    session_id=session_id,
    new_message=user_message
):
    event_count += 1
    author = event.author if event.author else 'system'
    
    print(f"\nEvent #{event_count}: {author}")
    print("-" * 70)
    
    # Show content parts
    if event.content and event.content.parts:
        for i, part in enumerate(event.content.parts):
            # Show function calls (what agent is requesting)
            if hasattr(part, 'function_call') and part.function_call:
                func_call = part.function_call
                print(f"\n  TOOL CALL: {func_call.name}")
                print(f"     ID: {func_call.id if hasattr(func_call, 'id') else 'N/A'}")
                if hasattr(func_call, 'args') and func_call.args:
                    args_dict = dict(func_call.args)
                    print(f"     Arguments:")
                    for key, value in args_dict.items():
                        print(f"       - {key}: {value}")
            
            # Show function responses (what tool returned)
            if hasattr(part, 'function_response') and part.function_response:
                func_resp = part.function_response
                print(f"\n  TOOL RESPONSE: {func_resp.name}")
                if hasattr(func_resp, 'response') and func_resp.response:
                    resp_dict = dict(func_resp.response)
                    print(f"     Full Response:")
                    for key, value in resp_dict.items():
                        # Pretty print the response
                        value_str = str(value)
                        if len(value_str) > 500:
                            # Format JSON nicely if it's JSON
                            try:
                                import json
                                parsed = json.loads(value_str)
                                value_str = json.dumps(parsed, indent=2)
                            except:
                                pass
                        print(f"       {key}:")
                        for line in value_str.split('\n'):
                            print(f"         {line}")
            
            # Show text content (agent's response)
            if hasattr(part, 'text') and part.text:
                print(f"\n  TEXT RESPONSE:")
                print("     " + "-" * 66)
                # Print full text with proper indentation
                for line in part.text.split('\n'):
                    print(f"     {line}")
                print("     " + "-" * 66)
    
    # Mark final responses
    if event.is_final_response():
        print(f"\n  FINAL RESPONSE from {author}")
        final_response = event

print("\n" + "="*70)

# Extract result from final response
result_data = {}
if final_response and final_response.content and final_response.content.parts:
    for part in final_response.content.parts:
        if hasattr(part, 'text') and part.text:
            json_start = part.text.find('{')
            json_end = part.text.rfind('}') + 1
            
            if json_start >= 0 and json_end > json_start:
                try:
                    result_data = json.loads(part.text[json_start:json_end])
                    break
                except json.JSONDecodeError as e:
                    logger.error(f"Failed to parse JSON: {e}")

total_duration = time.time() - orchestration_start

# Summary
print("\n" + "="*70)
print("EXECUTION SUMMARY")
print("="*70)
print(f"Total duration: {total_duration:.1f}s")
print(f"Total events: {event_count}")
print(f"Result keys: {len(result_data)}")
if result_data:
    print(f"\nResult structure:")
    for key in result_data.keys():
        value = result_data[key]
        if isinstance(value, list):
            print(f"   - {key}: list with {len(value)} items")
        elif isinstance(value, dict):
            print(f"   - {key}: dict with {len(value)} keys")
        else:
            print(f"   - {key}: {type(value).__name__}")
print("\nWORKFLOW COMPLETE!")
print("="*70)

2025-11-20 21:17:20,940 - INFO - google_adk.google.adk.models.google_llm - Sending out request, model: gemini-2.0-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
2025-11-20 21:17:20,941 - DEBUG - google_adk.google.adk.models.google_llm - 
LLM Request:
-----------------------------------------------------------
System Instruction:
Extract book metadata for the book provided by the user.
Use search_book to get book information and return it.

You are an agent. Your internal name is "book_metadata_researcher".
-----------------------------------------------------------
Config:
{'tools': [{}]}
-----------------------------------------------------------
Contents:
{"parts":[{"text":"Create a literary travel itinerary for \"gone with the wind\" by .\n\nExecute all steps and return the complete combined results."}],"role":"user"}
-----------------------------------------------------------
Functions:
search_book: {'title': {'type': <Type.STRING: 'STRING'>}, 'author': {'default':

EXECUTING WORKFLOW
Book: gone with the wind
Author: Unknown

USER REQUEST:
Create a literary travel itinerary for "gone with the wind" by .

Execute all steps and return the complete combined results.

Starting workflow execution...



2025-11-20 21:17:21,033 - DEBUG - httpcore.connection - start_tls.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x11b4749b0>
2025-11-20 21:17:21,033 - DEBUG - httpcore.http11 - send_request_headers.started request=<Request [b'POST']>
2025-11-20 21:17:21,034 - DEBUG - httpcore.http11 - send_request_headers.complete
2025-11-20 21:17:21,034 - DEBUG - httpcore.http11 - send_request_body.started request=<Request [b'POST']>
2025-11-20 21:17:21,035 - DEBUG - httpcore.http11 - send_request_body.complete
2025-11-20 21:17:21,035 - DEBUG - httpcore.http11 - receive_response_headers.started request=<Request [b'POST']>
2025-11-20 21:17:21,458 - DEBUG - httpcore.http11 - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b'application/json; charset=UTF-8'), (b'Vary', b'Origin'), (b'Vary', b'X-Origin'), (b'Vary', b'Referer'), (b'Content-Encoding', b'gzip'), (b'Date', b'Fri, 21 Nov 2025 02:17:21 GMT'), (b'Server', b'scaffolding on HTTPSe


Event #1: book_metadata_researcher
----------------------------------------------------------------------

  TEXT RESPONSE:
     ------------------------------------------------------------------
     I am sorry, I cannot fulfill this request. I can only extract book metadata.
     
     ------------------------------------------------------------------

  FINAL RESPONSE from book_metadata_researcher


2025-11-20 21:17:22,129 - DEBUG - httpcore.http11 - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b'application/json; charset=UTF-8'), (b'Vary', b'Origin'), (b'Vary', b'X-Origin'), (b'Vary', b'Referer'), (b'Content-Encoding', b'gzip'), (b'Date', b'Fri, 21 Nov 2025 02:17:22 GMT'), (b'Server', b'scaffolding on HTTPServer2'), (b'X-XSS-Protection', b'0'), (b'X-Frame-Options', b'SAMEORIGIN'), (b'X-Content-Type-Options', b'nosniff'), (b'Server-Timing', b'gfet4t7; dur=624'), (b'Alt-Svc', b'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'), (b'Transfer-Encoding', b'chunked')])
2025-11-20 21:17:22,131 - INFO - httpx - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-lite:generateContent "HTTP/1.1 200 OK"
2025-11-20 21:17:22,134 - DEBUG - httpcore.http11 - receive_response_body.started request=<Request [b'POST']>
2025-11-20 21:17:22,137 - DEBUG - httpcore.http11 - receive_response_body.complete
2025-11-20 21:

...


Event #8: author_formatter
----------------------------------------------------------------------

  TEXT RESPONSE:
     ------------------------------------------------------------------
     {
       "author_sites": [
         {
           "name": "Margaret Mitchell House",
           "type": "museum",
           "city": "Atlanta"
         },
         {
           "name": "Oakland Cemetery",
           "type": "cemetery",
           "city": "Atlanta"
         },
         {
           "name": "Atlanta History Center",
           "type": "history center",
           "city": "Atlanta"
         },
         {
           "name": "Kennesaw Mountain National Battlefield Park",
           "type": "battlefield park",
           "city": "Atlanta"
         },
         {
           "name": "The Wren's Nest",
           "type": "historic house",
           "city": "Atlanta"
         }
       ]
     }
     ------------------------------------------------------------------

  FINAL RESPONSE from au

2025-11-20 21:17:32,250 - DEBUG - httpcore.http11 - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b'application/json; charset=UTF-8'), (b'Vary', b'Origin'), (b'Vary', b'X-Origin'), (b'Vary', b'Referer'), (b'Content-Encoding', b'gzip'), (b'Date', b'Fri, 21 Nov 2025 02:17:32 GMT'), (b'Server', b'scaffolding on HTTPServer2'), (b'X-XSS-Protection', b'0'), (b'X-Frame-Options', b'SAMEORIGIN'), (b'X-Content-Type-Options', b'nosniff'), (b'Server-Timing', b'gfet4t7; dur=2198'), (b'Alt-Svc', b'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'), (b'Transfer-Encoding', b'chunked')])
2025-11-20 21:17:32,252 - INFO - httpx - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-lite:generateContent "HTTP/1.1 200 OK"
2025-11-20 21:17:32,252 - DEBUG - httpcore.http11 - receive_response_body.started request=<Request [b'POST']>
2025-11-20 21:17:32,254 - DEBUG - httpcore.http11 - receive_response_body.complete
2025-11-20 21


Event #9: landmark_formatter
----------------------------------------------------------------------

  TEXT RESPONSE:
     ------------------------------------------------------------------
     {
       "landmarks": [
         {
           "name": "Margaret Mitchell House",
           "city": "Atlanta, Georgia",
           "connection": "The apartment where Margaret Mitchell wrote most of the novel."
         },
         {
           "name": "Atlanta History Center",
           "city": "Atlanta, Georgia",
           "connection": "Offers exhibits related to the Civil War."
         },
         {
           "name": "Mary Mac's Tea Room",
           "city": "Atlanta, Georgia",
           "connection": "A landmark eatery serving Southern staples."
         },
         {
           "name": "Georgian Terrace Hotel",
           "city": "Atlanta, Georgia",
           "connection": "Where the \"Gone with the Wind\" premiere gala was held."
         },
         {
           "name": "Road to T

2025-11-20 21:17:36,772 - DEBUG - httpcore.http11 - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b'application/json; charset=UTF-8'), (b'Vary', b'Origin'), (b'Vary', b'X-Origin'), (b'Vary', b'Referer'), (b'Content-Encoding', b'gzip'), (b'Date', b'Fri, 21 Nov 2025 02:17:36 GMT'), (b'Server', b'scaffolding on HTTPServer2'), (b'X-XSS-Protection', b'0'), (b'X-Frame-Options', b'SAMEORIGIN'), (b'X-Content-Type-Options', b'nosniff'), (b'Server-Timing', b'gfet4t7; dur=4292'), (b'Alt-Svc', b'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'), (b'Transfer-Encoding', b'chunked')])
2025-11-20 21:17:36,774 - INFO - httpx - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-lite:generateContent "HTTP/1.1 200 OK"
2025-11-20 21:17:36,777 - DEBUG - httpcore.http11 - receive_response_body.started request=<Request [b'POST']>
2025-11-20 21:17:36,780 - DEBUG - httpcore.http11 - receive_response_body.complete
2025-11-20 21


Event #11: trip_composer
----------------------------------------------------------------------

  TEXT RESPONSE:
     ------------------------------------------------------------------
     {
       "cities": [
         {
           "name": "Atlanta",
           "country": "USA",
           "days_suggested": 3,
           "overview": "Immerse yourself in the heart of the Gone With The Wind story. Atlanta offers a deep dive into the historical setting, the life of Margaret Mitchell, and the ambiance of the Civil War and Reconstruction era.",
           "stops": [
             {
               "name": "Margaret Mitchell House",
               "type": "museum",
               "reason": "Visit the apartment where Margaret Mitchell wrote 'Gone With The Wind'.",
               "time_of_day": "morning",
               "notes": "Allow 2-3 hours for the tour and exhibits."
             },
             {
               "name": "Atlanta History Center",
               "type": "history center",


### Step 5: Display Results

Parse and display the collected results from all agents.

## 📚 Implementation: Pydantic & ADK v1.19+

### What's Implemented

This notebook demonstrates **Pydantic-validated multi-agent orchestration** using:
- **Pydantic Models** for type-safe data structures
- **ADK v1.19+ native `output_schema`** for automatic validation
- **Single-stage agents** with both tools and Pydantic (bug fixed in v1.19+)
- **Conversation history** for implicit agent communication

### Key Benefits

✅ **Type Safety** - Pydantic validates all outputs automatically  
✅ **Clear Contracts** - Each agent's output structure is explicitly defined  
✅ **Simpler Architecture** - Single-stage agents (no two-stage workaround needed)  
✅ **Better Debugging** - Validation errors caught early with field-level details  
✅ **State Tracking** - All outputs saved to session state via `output_key`  

### Agent Architecture

```
SequentialAgent (overall_workflow)
├─ book_metadata_agent [tools: google_books_tool, schema: BookMetadata]
├─ book_context_agent [tools: google_search, schema: BookContext]  
├─ ParallelAgent (parallel_discovery) ⚡ CONCURRENT
│  ├─ city_agent [tools: google_search, schema: CityDiscovery]
│  ├─ landmark_agent [tools: google_search, schema: LandmarkDiscovery]
│  └─ author_agent [tools: google_search, schema: AuthorSites]
└─ trip_composer_agent [schema: TripItinerary]
```

Each agent:
- Uses `tools` for data gathering (Google Search, Google Books API)
- Uses `output_schema` for Pydantic validation
- Uses `output_key` to save validated output to session state
- Accesses previous outputs via conversation history

### Pydantic Models Used

| Model | State Key | Agent | Fields |
|-------|-----------|-------|--------|
| BookMetadata | `book_metadata` | book_metadata_agent | book_title, author, description, etc. |
| BookContext | `book_context` | book_context_agent | primary_locations, time_period, themes |
| CityDiscovery | `city_discovery` | city_agent | cities (list of CityInfo) |
| LandmarkDiscovery | `landmark_discovery` | landmark_agent | landmarks (list of LandmarkInfo) |
| AuthorSites | `author_sites` | author_agent | author_sites (list of AuthorSiteInfo) |
| TripItinerary | `final_itinerary` | trip_composer_agent | cities (list of CityPlan), summary_text |

### ADK v1.19+ Improvements

**Previous (v1.18.0):** Agents with both `output_schema` + `tools` could enter infinite loops  
**Workaround:** Two-stage pattern (tool agent → formatting agent)

**Now (v1.19+):** Bug fixed! Can use `output_schema` + `tools` together safely  
**Simplified:** Single-stage agents with both capabilities

**Example:**
```python
# ADK v1.19+ - Single stage works perfectly
agent = LlmAgent(
    tools=[google_search],
    output_schema=CityDiscovery,  # Pydantic model
    output_key="city_discovery"
)
```

### Communication Pattern

Agents use **implicit communication** via conversation history:
- Each agent sees all previous agent outputs in the message stream
- Instructions reference "conversation history above" to access context
- Simple, reliable, and works naturally with LLMs
- Pydantic adds explicit validation on outputs

This hybrid approach combines:
- **Implicit input** (conversation history) - natural for LLMs
- **Explicit output** (Pydantic validation) - type safety and reliability

In [110]:
# The result_data IS the TripItinerary (no 'itinerary' wrapper when using Pydantic)
# Keys are directly: 'cities' and 'summary_text'

print("="*70)
print("FINAL RESULT")
print("="*70)

if not result_data:
    print("\nNo result data found")
else:
    print(f"\nResult keys: {list(result_data.keys())}")
    
    # Get cities directly from result_data
    cities_list = result_data.get('cities', [])
    
    if cities_list:
        print(f"\nTRAVEL ITINERARY - {len(cities_list)} City Plan(s)")
        print("="*70)
        
        for i, city_plan in enumerate(cities_list, 1):
            print(f"\nCity #{i}: {city_plan.get('name')}, {city_plan.get('country')}")
            print(f"   Days suggested: {city_plan.get('days_suggested', 1)}")
            
            if city_plan.get('overview'):
                print(f"\n   Overview:")
                print(f"   {city_plan.get('overview')}")
            
            stops = city_plan.get('stops', [])
            if stops:
                print(f"\n   Stops ({len(stops)}):")
                print("   " + "-"*66)
                
                for j, stop in enumerate(stops, 1):
                    tod = stop.get('time_of_day', 'unknown').upper()
                    stop_type = stop.get('type', 'other')
                    
                    print(f"\n   {j}. {stop.get('name')}")
                    print(f"      Time: {tod}")
                    print(f"      Type: {stop_type}")
                    
                    if stop.get('reason'):
                        print(f"      Connection to book:")
                        print(f"         {stop.get('reason')}")
                    
                    if stop.get('notes'):
                        print(f"      Notes:")
                        print(f"         {stop.get('notes')}")
                
                print("   " + "-"*66)
            
            print()  # Blank line between cities
        
        # Summary
        summary_text = result_data.get('summary_text')
        if summary_text:
            print("="*70)
            print("TRIP SUMMARY")
            print("="*70)
            print(f"\n{summary_text}")
            print()
    else:
        print("\nNo cities found in result")

# Statistics
print("\n" + "="*70)
print("STATISTICS")
print("="*70)

if cities_list:
    total_cities = len(cities_list)
    total_stops = sum(len(city.get('stops', [])) for city in cities_list)
    total_days = sum(city.get('days_suggested', 0) for city in cities_list)
    
    print(f"Total cities: {total_cities}")
    print(f"Total stops: {total_stops}")
    print(f"Total days: {total_days}")
    
    # Count stop types
    stop_types = {}
    for city in cities_list:
        for stop in city.get('stops', []):
            stop_type = stop.get('type', 'other')
            stop_types[stop_type] = stop_types.get(stop_type, 0) + 1
    
    if stop_types:
        print(f"\nStops by type:")
        for stop_type, count in sorted(stop_types.items(), key=lambda x: x[1], reverse=True):
            print(f"   • {stop_type}: {count}")
else:
    print("No itinerary data available")

print("\n✅ Display complete!")
print("="*70)

FINAL RESULT

Result keys: ['cities', 'summary_text']

TRAVEL ITINERARY - 3 City Plan(s)

City #1: Atlanta, USA
   Days suggested: 3

   Overview:
   Immerse yourself in the heart of the Gone With The Wind story. Atlanta offers a deep dive into the historical setting, the life of Margaret Mitchell, and the ambiance of the Civil War and Reconstruction era.

   Stops (5):
   ------------------------------------------------------------------

   1. Margaret Mitchell House
      Time: MORNING
      Type: museum
      Connection to book:
         Visit the apartment where Margaret Mitchell wrote 'Gone With The Wind'.
      Notes:
         Allow 2-3 hours for the tour and exhibits.

   2. Atlanta History Center
      Time: AFTERNOON
      Type: history center
      Connection to book:
         Explore Civil War exhibits and gain context for the historical events.
      Notes:
         Plan for 3-4 hours.

   3. Mary Mac's Tea Room
      Time: EVENING
      Type: restaurant
      Connection t